## 수익률 표준편차

In [28]:
import pandas as pd

# cp949 또는 euc-kr 인코딩 시도
df = pd.read_csv('../data/주가수익률_표준편차.csv', encoding='cp949')
df.drop("Unnamed: 0", axis=1, inplace=True)
# 만약 cp949에서 에러가 발생하면 utf-8로 시도해보세요
# df = pd.read_csv('data/주가수익률_표준편차.csv', encoding='utf-8')
df.to_csv("주가수익률test.csv", encoding='utf-8', index=False)

In [30]:
df = df[df['기간구간'] != '기타']
df

,회사명,공모가 (원),회계년도,종가(원),수익률,일자순번,기간구간,수익률_표준편차
0,AJ네트웍스,34300,2015-08-21,35500,0.0350,1,1~5일,0.089380
1,AJ네트웍스,34300,2015-08-24,32400,-0.0873,2,1~5일,0.089380
2,AJ네트웍스,34300,2015-08-25,31100,-0.0401,3,1~5일,0.089380
3,AJ네트웍스,34300,2015-08-26,35450,0.1399,4,1~5일,0.089380
4,AJ네트웍스,34300,2015-08-27,37850,0.0677,5,1~5일,0.089380
...,...,...,...,...,...,...,...,...
741814,힘스,20000,2017-08-10,27100,-0.0356,16,16~20일,0.020914
741815,힘스,20000,2017-08-11,26800,-0.0111,17,16~20일,0.020914
741816,힘스,20000,2017-08-14,26800,0.0000,18,16~20일,0.020914
741817,힘스,20000,2017-08-16,27400,0.0224,19,16~20일,0.020914


In [33]:
df[df['기간구간'] != '기타'].to_csv("주가수익률test.csv", encoding='utf-8', index=False)

In [15]:
rename_map = {
    "1~5일": "1~5일 수익률 표준편차",
    "6~10일": "6~10일 수익률 표준편차",
    "11~15일": "11~15일 수익률 표준편차",
    "16~20일": "16~20일 수익률 표준편차",
}

pivot = (
    df.pivot_table(index="회사명",
                   columns="기간구간",
                   values="수익률_표준편차",
                   aggfunc="mean")          # 중복 → 평균
      .rename(columns=rename_map)
)

ordered_cols = list(rename_map.values())
wide_df = pivot[ordered_cols].reset_index()

# 저장
wide_csv_path = "주가수익률test_WIDE.csv"
wide_df.to_csv(wide_csv_path, index=False, encoding="utf-8-sig")

print("CSV 파일 저장 위치:")
print(" - 변환본 :", wide_csv_path)

CSV 파일 저장 위치:
 - 변환본 : 주가수익률test_WIDE.csv


In [19]:
df = pd.read_csv("../data/주가수익률_final.csv", index_col=0)
df.isna().sum()

회사명                0
1~5일 수익률 표준편차      0
6~10일 수익률 표준편차     2
11~15일 수익률 표준편차    6
16~20일 수익률 표준편차    6
stock_code         0
dtype: int64

## 노션데이터 수정

In [ ]:
gi = pd.read_csv("gi.csv")
gi.drop("Unnamed: 0", axis=1, inplace=True)




        회사명  stock_code         상장일
0     바이오노트      377740  2022-12-22
1        쏘카      403550  2022-08-22
2   수산인더스트리      126720  2022-08-01
3  LG에너지솔루션      373220  2022-01-27
4     카카오페이      377300  2021-11-03
